In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hackhathon-repik-ai/train.csv
/kaggle/input/hackhathon-repik-ai/test.csv
/kaggle/input/hackhathon-repik-ai/damage_classification_model.h5
/kaggle/input/hackhathon-repik-ai/test/test/7981.jpg
/kaggle/input/hackhathon-repik-ai/test/test/10304.jpg
/kaggle/input/hackhathon-repik-ai/test/test/11925.jpg
/kaggle/input/hackhathon-repik-ai/test/test/11378.jpg
/kaggle/input/hackhathon-repik-ai/test/test/10730.jpg
/kaggle/input/hackhathon-repik-ai/test/test/7737.jpg
/kaggle/input/hackhathon-repik-ai/test/test/10597.jpg
/kaggle/input/hackhathon-repik-ai/test/test/10054.jpg
/kaggle/input/hackhathon-repik-ai/test/test/7966.jpg
/kaggle/input/hackhathon-repik-ai/test/test/10924.jpg
/kaggle/input/hackhathon-repik-ai/test/test/9960.jpg
/kaggle/input/hackhathon-repik-ai/test/test/9131.jpg
/kaggle/input/hackhathon-repik-ai/test/test/9620.jpg
/kaggle/input/hackhathon-repik-ai/test/test/9062.jpg
/kaggle/input/hackhathon-repik-ai/test/test/10213.jpg
/kaggle/input/hackhathon-repik-ai/test/test/8

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load CSV file

df = pd.read_csv('/kaggle/input/hackhathon-repik-ai/train.csv')
# Convert integer labels to strings
df['label'] = df['label'].astype(str)

# Split dataset into training and testing sets
train_df, val_df = train_test_split(df, test_size=0.3, random_state=42)

# Create ImageDataGenerator for data augmentation and normalization
datagen = ImageDataGenerator(rescale=1./255, 
                             shear_range=0.2, 
                             zoom_range=0.2, 
                             horizontal_flip=True)



# Specify the directory containing all images
img_directory = '/kaggle/input/hackhathon-repik-ai/train/train'


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Create data generators
img_size = (224, 224)  # VGG16 input size
batch_size = 32
train_generator = datagen.flow_from_dataframe(dataframe=train_df, 
                                              directory=img_directory, 
                                              x_col='filename', y_col='label', 
                                              class_mode='categorical', 
                                              target_size=img_size, 
                                              batch_size=batch_size)
val_generator = datagen.flow_from_dataframe(dataframe=val_df, 
                                            directory=img_directory, 
                                            x_col='filename', 
                                            y_col='label', 
                                            class_mode='categorical', 
                                            target_size=img_size, 
                                            batch_size=batch_size)

Found 32613 validated image filenames belonging to 6 classes.
Found 13977 validated image filenames belonging to 6 classes.


# NasNetLarge

In [13]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.applications import NASNetLarge

# NASNetLarge
base_model_nasnet = NASNetLarge(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model_nasnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
predictions_nasnet = Dense(6, activation='softmax')(x)
model_nasnet = Model(inputs=base_model_nasnet.input, outputs=predictions_nasnet)

# Compile the model
model_nasnet.compile(optimizer=RMSprop(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Learning rate scheduler
def lr_scheduler(epoch, lr):
    return lr * 0.9  # Adjust the factor as needed

learning_rate_scheduler = LearningRateScheduler(lr_scheduler)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train NASNetLarge
model_nasnet.fit(train_generator, epochs=10, validation_data=val_generator, callbacks=[learning_rate_scheduler, early_stopping])

# Evaluate NASNetLarge
eval_result_nasnet = model_nasnet.evaluate(val_generator)
print(f"NASNetLarge Test Accuracy: {eval_result_nasnet[1]*100:.2f}%")


343610240/343610240 [==============================] - 1s 0us/step
Epoch 1/10
1020/1020 [==============================] - 1831s 2s/step - loss: 0.1712 - accuracy: 0.9409 - val_loss: 1.4635 - val_accuracy: 0.8910 - lr: 9.0000e-05
Epoch 2/10
1020/1020 [==============================] - 1676s 2s/step - loss: 0.0155 - accuracy: 0.9955 - val_loss: 0.2504 - val_accuracy: 0.9833 - lr: 8.1000e-05
Epoch 3/10
1020/1020 [==============================] - 1673s 2s/step - loss: 0.0082 - accuracy: 0.9976 - val_loss: 0.8507 - val_accuracy: 0.9488 - lr: 7.2900e-05
Epoch 4/10
1020/1020 [==============================] - 1675s 2s/step - loss: 0.0029 - accuracy: 0.9990 - val_loss: 0.2206 - val_accuracy: 0.9820 - lr: 6.5610e-05
Epoch 5/10
1020/1020 [==============================] - 1672s 2s/step - loss: 0.0022 - accuracy: 0.9995 - val_loss: 0.2652 - val_accuracy: 0.9758 - lr: 5.9049e-05
Epoch 6/10
1020/1020 [==============================] - 1678s 2s/step - loss: 0.0016 - accuracy: 0.9996 - val_loss: 0.

In [14]:
# Save the trained model
model_nasnet.save('model_nasnet_damage_classification_model1.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
import os
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Set your data paths
test_dir = '/kaggle/input/hackhathon-repik-ai/test/test'
test_csv_path = '/kaggle/input/hackhathon-repik-ai/test.csv'
model_path = 'model_nasnet_damage_classification_model1.h5'  # Replace with the path to your trained model

# Load the trained model
model = load_model(model_path)

# Read the test data CSV file
test_df = pd.read_csv(test_csv_path)

# Create an empty list to store the predicted labels
predicted_labels = []

# Iterate through each row in the test dataframe
for index, row in test_df.iterrows():
    # Load and preprocess the test image
    img_path = os.path.join(test_dir, row['filename'])
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize the pixel values
    img_array = img_array.reshape((1, 224, 224, 3))

    # Make a prediction using the model
    prediction = model.predict(img_array)
    predicted_label = int(prediction.argmax(axis=-1)) + 1  # Assuming class indices start from 1
    predicted_labels.append(predicted_label)

# Add the predicted labels to the test dataframe
test_df['label'] = predicted_labels

# Save the dataframe with predicted labels to a new CSV file
test_df.to_csv('model_nasnet_predicted_test.csv', index=False)
print("predicted CSV successfully")

1/1 [==============================] - 0s 44ms/step
predicted CSV successfully
